flowchart LR
    subgraph Azure
        EG(Event Grid) --> JM(Job Manager)
        JM --> SB[Service Bus Queue]
        SB --> W1[Document Processor Worker]
        W1 --> JSONBlob[Processed JSON in Blob]
        W1 --> SB2[Service Bus Queue - Embedding]
        SB2 --> W2[Embedding Worker]
        W2 --> MosaicDB[(MosaicDB Vector Store)]
    end

    U(User) -->|Query + Filters| SearchAPI
    SearchAPI --> MosaicDB

flowchart TD
    A[Azure Blob Storage] -- New Document --> B[Event Grid]
    B -- Event Trigger --> C[job_manager (Event Grid Trigger)]
    C -->|Blob info JSON| D[Azure Service Bus Queue]
    D --> E[worker (Service Bus Trigger)]
    E -->|Choose processor| F1[pdf_processor.py]
    E -->|Choose processor| F2[docx_processor.py]
    F1 --> G[Blob JSON Upload (processed text + metadata)]
    F2 --> G
    G --> H[chunk_embed_processor.py]
    H --> I[MosaicDB Vector Store]
    I --> J[Search API - Vector + Metadata filter]


project_root/
│
├── main.py
├── config.py
├── config.yaml
│
├── db/
│   ├── base.py
│   ├── crud.py
│   ├── models.py
│   ├── session.py
│
├── functions/
│   ├── diverter_function.py
│   ├── processor_pdf.py
│   ├── processor_docx.py
│   └── ...
│
├── utils/
│   ├── blob_utils.py
│   ├── embedding_utils.py
│   ├── logging_utils.py
│   ├── servicebus_utils.py
│
├── job_manager/
│   ├── __init__.py   ← Azure Function entry point
│   ├── function.json ← Azure Function trigger config
│
└── requirements.txt


In [ ]:
# If run locally for testing ------------------------------------------------
if __name__ == "__main__":
    # example local test (simulate Event Grid payload)
    sample_event = [{
        "eventType": "Microsoft.Storage.BlobCreated",
        "subject": "/blobServices/default/containers/test-container/blobs/test-folder/sample.pdf",
        "data": {
            "api": "PutBlob",
            "clientRequestId": "xxx",
            "requestId": "yyy",
            "eTag": "0x8D8123456789AB",
            "contentType": "application/pdf",
            "contentLength": 12345,
            "blobType": "BlockBlob",
            "url": "https://account.blob.core.windows.net/test-container/test-folder/sample.pdf"
        }
    }]

    print("Simulating Event Grid event -> job_manager")
    res = http_eventgrid_handler(sample_event)
    print(json.dumps(res, indent=2))